<a target="_blank" href="https://colab.research.google.com/github/retowuest/uio-dl-2024/blob/main/Notebooks/nb-4.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Deep Learning for Social Scientists

### PhD Course, University of Bergen

### **Notebook 4:**<br>Transformers

### Table of Contents
* [Introduction](#section_1)
* [Loading the Data](#section_2)
* [Loading and Fine-Tuning a Pre-Trained BERT Model](#section_3)

### Introduction <a class="anchor" id="section_1"></a>

In this notebook, our goal is to fine-tune a BERT model for sentiment classification in PyTorch. We will use the `transformers` [Python library](https://huggingface.co/docs/transformers/index) provided by [Hugging Face](https://huggingface.co/), which includes a number of pre-trained models that are ready for fine-tuning.

We will use as our use case the IMDb movie review data set and fine-tune the distilled BERT model (`DistilBERT`) to perform sentiment classification. `DistilBERT` is a lightweight transformer model created by distilling a pre-trained BERT base model. The original uncased BERT base model contains over 110 million parameters. According to Hugging Face (see quote below), `DistilBERT` has 40% fewer parameters and runs 60% faster while preserving 95% of BERT's performance on the GLUE language understanding benchmark.

---

Quote from https://huggingface.co/docs/transformers/model_doc/distilbert:

> DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. It has 40% less parameters than *google-bert/bert-base-uncased*, runs 60% faster while preserving over 95% of BERT's performances as measured on the GLUE language understanding benchmark.

---

### Loading the Data <a class="anchor" id="section_1"></a>

We will begin by loading the required packages.

In [1]:
# Import packages
import gzip
import shutil
import time

import pandas as pd
import requests
import torch
import torch.nn.functional as F
import torchtext

import transformers
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification

/Users/retowuest/learning-workshops/uib-dl-2024/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Next, we specify some general settings (number of epochs we use for training the model, device specification, and the random seed).

In [2]:
#torch.backends.cudnn.deterministic = True (for reproducibility when using NVIDIA CUDA Deep Neural Network (cuDNN))
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)
DEVICE = torch.device("cpu")

NUM_EPOCHS = 3

Next, we will fetch the compressed IMDb movie review dataset (http://ai.stanford.edu/~amaas/data/sentiment/) for positive-negative sentiment classification, unzip it, and write it into a CSV-formatted file.

In [3]:
url = "https://github.com/rasbt/machine-learning-book/raw/main/ch08/movie_data.csv.gz"
filename = url.split("/")[-1]

with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)

with gzip.open("movie_data.csv.gz", "rb") as f_in:
    with open("movie_data.csv", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out) # copy content from source file to destination file

Check if the data set looks okay.

In [4]:
# Load data into a Pandas DataFrame and print first few rows
df = pd.read_csv("movie_data.csv")
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [5]:
# Print dimensions of dataframe
df.shape

(50000, 2)

The next step is to split data set into training, validation, and test sets. We use 70% (or 35,000 examples) of the data for training, 10% (or 5,000 examples) for validation, and the remaining 20% (or 10,000 examples) for testing.

In [6]:
# Split data into training, validation, and test sets
train_texts = df.iloc[:35000]["review"].values
train_labels = df.iloc[:35000]["sentiment"].values

valid_texts = df.iloc[35000:40000]["review"].values
valid_labels = df.iloc[35000:40000]["sentiment"].values

test_texts = df.iloc[40000:]["review"].values
test_labels = df.iloc[40000:]["sentiment"].values

Next, we will tokenize the texts into individual word tokens using the tokenizer implementation from the pre-trained model class.

In [7]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased", model_max_length=512)

In [8]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
valid_encodings = tokenizer(list(valid_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

Finally, we create a class called `IMDbDataset` and create the data loaders (the encodings store a lot of information about the tokenized texts; with the dictionary in the `__getitem__` method defined below, we extract only the relevant information).

In [10]:
# Create class IMDbDataset
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = IMDbDataset(train_encodings, train_labels)
valid_dataset = IMDbDataset(valid_encodings, valid_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [11]:
# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

### Loading and Fine-Tuning a Pre-Trained BERT Model <a class="anchor" id="section_3"></a>

We first load the pre-trained BERT model ("uncased" means that the model does not distinguish between upper- and lower-case letters).

In [12]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased") # this specifies the downstream task for which we want to fine-tune the model
model.to(DEVICE)
model.train();

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


To train (fine-tune) the model, we will use the `Trainer` API provided by `Hugging Face`.

In [13]:
# Import Trainer and TrainingArguments from transformers
from transformers import Trainer, TrainingArguments

# Specify optimization algorithm
optim = torch.optim.Adam(model.parameters(), lr=5e-5)

# Specify training arguments
# (directories for output and logs, number of epochs, batch sizes)
training_args = TrainingArguments(
    output_dir="./results", 
    num_train_epochs=3,     
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,   
    logging_dir="./logs",
    logging_steps=10,
)

# Pass TrainingArguments settings to the Trainer class to instantiate a new trainer object
trainer = Trainer(
    model=model, # the model to be fine-tuned
    args=training_args, # training arguments specified above
    train_dataset=train_dataset, # training set
    optimizers=(optim, None) # optim and learning rate schedule
)

We can now train the model by calling the `trainer.train` method (we will use this method shortly).

The `Trainer` API only shows the training loss and does not provide model evaluation. Therefore, to evaluate the model, we define an evaluation function.

In [14]:
# Import load_metrics and numpy
from datasets import load_metric
import numpy as np

# Define metric
metric = load_metric("accuracy")

# Define evaluation function
# (function operates on the model's test predictions as logits, which is the default output of the model, and the labels)
def compute_metrics(eval_pred):
    logits, labels = eval_pred # logits are a numpy array, not pytorch tensor
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(
               predictions=predictions, references=labels)

/var/folders/99/f1dvt7lj4777dzt7yj3hz2pr0000gn/T/ipykernel_75667/2031210347.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/Users/retowuest/learning-workshops/uib-dl-2024/.venv/lib/python3.12/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [15]:
# Run trainer again, this time including test set and compute_metrics
optim = torch.optim.Adam(model.parameters(), lr=5e-5)

training_args = TrainingArguments(
    output_dir="./results", 
    num_train_epochs=3,     
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,   
    logging_dir="./logs",
    logging_steps=10
)

trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    optimizers=(optim, None) # optimizer and learning rate scheduler
)

In [16]:
# Train model by calling trainer.train method
start_time = time.time()
trainer.train()
print(f"Total Training Time: {(time.time() - start_time)/60:.2f} min")

  0%|          | 0/6564 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  0%|          | 10/6564 [00:16<2:15:53,  1.24s/it]

{'loss': 0.6931, 'grad_norm': 1.5085525512695312, 'learning_rate': 4.9923826934795856e-05, 'epoch': 0.0}


  0%|          | 20/6564 [00:28<2:14:05,  1.23s/it]

{'loss': 0.6482, 'grad_norm': 4.474645614624023, 'learning_rate': 4.984765386959172e-05, 'epoch': 0.01}


  0%|          | 30/6564 [00:40<2:12:24,  1.22s/it]

...

{'loss': 0.0949, 'grad_norm': 0.06539382040500641, 'learning_rate': 1.0664229128580134e-07, 'epoch': 2.99}


100%|█████████▉| 6560/6564 [2:22:00<00:04,  1.22s/it]

{'loss': 0.0012, 'grad_norm': 0.04037441685795784, 'learning_rate': 3.046922608165753e-08, 'epoch': 3.0}


100%|██████████| 6564/6564 [2:22:04<00:00,  1.30s/it]

{'train_runtime': 8524.5142, 'train_samples_per_second': 12.317, 'train_steps_per_second': 0.77, 'train_loss': 0.14979129153661916, 'epoch': 3.0}
Total Training Time: 142.08 min


After training has completed (142min), we can call `trainer.evaluate` to obtain the model performance on the test set.

In [17]:
print(trainer.evaluate())

100%|██████████| 625/625 [03:57<00:00,  2.63it/s]

{'eval_loss': 0.3030503988265991, 'eval_accuracy': 0.9362, 'eval_runtime': 239.074, 'eval_samples_per_second': 41.828, 'eval_steps_per_second': 2.614, 'epoch': 3.0}


The evaluation accuracy is around 94%.